# PISA of Ising MIRT Ising Theta($\theta_2$) and Q reSample

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data preprocessing

In [ ]:
raw_df = pd.read_spss("CY07_MSU_STU_COG_testlet.sav")

In [ ]:
# All the Process was Carried Out With Pandas

fil1 = raw_df.iloc[:, 13:65]      # Clear All the Information Except Responses

fil2 = fil1.replace(['Full credit', '1 - Full credit', '2 - Full credit', 'No credit', '0 - No credit'], [1, 1, 1, 0, 0])
# Invert All Responses in terms of Binary Codes(1: Correct, 0: Incorrect)

fil3 = fil2.drop('CM955Q03S', axis=1)    # Clear the Item of Multiple Choices
fil4 = fil3.dropna(how='all')            # Clear All Students of No Responses & Clear All Items of No Responses

In [ ]:
# DataFrame to Numpy
num_np = fil4.to_numpy()

# The Conversion Process to avoid 'divided by zero' error
scarub_np = np.where(num_np == 1, 0.99, num_np)
scourge_np = np.where(scarub_np == 0, 0.01, scarub_np)
num_df = scourge_np                                     # Caution!! num_df is of numpy, not of pandas!!

# print(num_df)

num_dfdf = pd.DataFrame(num_df)                         # num_dfdf is, at last, of pandas!
p_solves = num_dfdf.notnull().sum(1)                    # count the number of responses regardless of NaN

# Data shape
rows, columns = num_df.shape
# print(rows, columns)


## Selection of Responses for the Test Set

In [ ]:
from collections import Counter
import random
import math

In [ ]:
train_gagongs = []
test_gagongs = []
num_iter = 10

for i in range(num_iter):
    
    # 1st, input the shared data from UIRT.
    train_carrier_df = pd.read_csv("share_traindf0.1_{0}_1518_0701.csv".format(i+1))
    test_carrier_df = pd.read_csv("share_testdf0.1_1518_{0}_0701.csv".format(i+1))
    
    train_carrier_df.drop(['Unnamed: 0'], axis=1, inplace=True)
    test_carrier_df.drop(['Unnamed: 0'], axis=1, inplace=True)
    
    train_gagongs.append(train_carrier_df)
    test_gagongs.append(test_carrier_df)


## List of Functions of Requirement

In [ ]:
# This function is for the comparison with the reference data.
# Only the train_gagong_let is given in pandas.
def expect_model(alpha_let, alpha2le, d_let, theta_let, theta2le, train_gagong_let):
    
    gagong_let = train_gagong_let.to_numpy()
    exponet_1 = alpha_let * theta_let
    exponet_2 = alpha2le * theta2le
    
    before_nan = np.exp(exponet_1 + exponet_2 - d_let)/(1+np.exp(exponet_1 + exponet_2 - d_let))
    after_nan = before_nan.copy()
    
    # Reflection of NaN data
    for n in range(before_nan.shape[0]):
        for m in range(before_nan.shape[1]):
            if np.isnan(gagong_let[n][m]):
                after_nan[n][m] = np.nan
    
    # The Conversion Process to avoid 'divided by zero' error
    scarub = np.where(after_nan > 0.99, 0.99, after_nan)
    scourge = np.where(scarub < 0.01, 0.01, scarub)
    model_result = scourge
    
    return model_result                  # The result is yielded in numpy form.


In [ ]:
# Common parts of chain rule of D_KL's derivative.
# Only the train_gagong_let is given in pandas.
def preprocess_diff(alpha_let, alpha2le, d_let, theta_let, theta2le, train_gagong_let):
    
    gagong_let = train_gagong_let.to_numpy()
    p_imu = expect_model(alpha_let, alpha2le, d_let, theta_let, theta2le, train_gagong_let)  # from the model
    q_imu = gagong_let.copy()                                                                # from the reference data

    # 바로 p와 q 조합
    KLD_common = p_imu - q_imu           # KLD represents the initial of 'K'ullback-'L'eibler 'D'ivergence.
    
    return KLD_common                   # The result is yielded in 2D numpy form.

In [ ]:
# the function to update alpha
# Only the train_gagong_let is given in pandas.
def set_alpha(alpha_let, alpha2le, d_let, theta_let, theta2le, train_gagong_let):
    
    # Loading the common part
    common_unit = preprocess_diff(alpha_let, alpha2le, d_let, theta_let, theta2le, train_gagong_let)
    
    # Calculation Start
    delta_matrix = theta_let * common_unit                         # before summation (formed in numpy)
    
    # Get rid of Missing Data
    dmatrix_df = pd.DataFrame(delta_matrix)                        
    dmatrix_fna = dmatrix_df.fillna(0)                             
    delta_matrix2 = dmatrix_fna.to_numpy()
    
    delta_alphak = delta_matrix2.sum(axis=0, keepdims = True)      # summation in terms of examinees
    
    alpha_med = alpha_let - A * delta_alphak             # alpha update by means of Gradient Descent
    alpha_result = alpha_med
    
    return alpha_result                                 # The result is yielded in numpy form.

In [ ]:
# the function to update d
# Only the train_gagong_let is given in pandas.
def set_delta(alpha_let, alpha2le, d_let, theta_let, theta2le, train_gagong_let):
    
    # Loading the common part
    common_unit = preprocess_diff(alpha_let, alpha2le, d_let, theta_let, theta2le, train_gagong_let)
    
    # Calculation Start
    delta_matrix = (-1) * common_unit                         # before summation (formed in numpy)
    
    # Get rid of Missing Data
    dmatrix_df = pd.DataFrame(delta_matrix)                        
    dmatrix_fna = dmatrix_df.fillna(0)                             
    delta_matrix2 = dmatrix_fna.to_numpy()
    
    delta_dtak = delta_matrix2.sum(axis=0, keepdims = True)      # summation in terms of examinees
    
    d_med = d_let - A * delta_dtak                            # d update by means of Gradient Descent
    d_result = d_med - np.mean(d_med)                         # standardization of d

    return d_result                                          # The result is yielded in numpy form.

In [ ]:
# the function to update theta_1
# Only the train_gagong_let is given in pandas.
def update_theta(alpha_let, alpha2le, d_let, theta_let, theta2le, train_gagong_let):
    
    # Loading the common part
    common_unit = preprocess_diff(alpha_let, alpha2le, d_let, theta_let, theta2le, train_gagong_let)
    
    # Calculation Start
    delta_matrix = alpha_let * common_unit                         # before summation (formed in numpy)
    
    # Get rid of Missing Data
    dmatrix_df = pd.DataFrame(delta_matrix)                        
    dmatrix_fna = dmatrix_df.fillna(0)                             
    delta_matrix2 = dmatrix_fna.to_numpy()
    
    delta_thetak = delta_matrix2.sum(axis=1, keepdims = True)   # summation in terms of items
        
    theta_update = theta_let - A * delta_thetak                 # theta_1 update by means of Gradient Descent

    return theta_update                                         # The result is yielded in numpy form.

### List of Functions for Updating $\theta_{2}$ Only

In [ ]:
from IPython.display import Image

In [ ]:
Image("theta2_transform.png")

In [ ]:
Image("theta2_update.png")

In [ ]:
# Both samjin_data and Q_let are of numpy.    'samjin' means 'trinary' in Korean.
def Shell_gagong(samjin_data, Q_let):
    
    num_gagong = samjin_data.copy()
    rows_let = num_gagong.shape[0]
    columns_let = num_gagong.shape[1]

    shell_list = []

    for i in range(rows_let):
        garo_pre = num_gagong[i, :]                      # response vector(Y) of !D. 'garo' means 'horizon' in Korean.
        garo_T = np.reshape(garo_pre, (columns_let, 1))  # vertical form
        sero = garo_T.copy()                             # 'sero' means 'the vertical' in Korean.
        garo = np.transpose(garo_T)
        shell_rough = sero * garo                    # 2D matrix of all the combination of Y_i Y_j (symmetric)

        carrier = Q_let * shell_rough                # 2D matrix with intensity Q
        np.fill_diagonal(carrier, 0)                 # off-diagonal

        shell_list.append(carrier)

    shell_result = np.array(shell_list)          # The result is yielded in 3-Rank Tensor
    
    return shell_result

In [ ]:
# The function to generate ingredient for pseudo-probability from Ising Hamiltonian
# Gagong_data is of pandas and Q_let is of numpy.
def answer_covari_bfsum(gagong_data, Q_let):

    num_gagong_bf = gagong_data.to_numpy()
    rows_let = num_gagong_bf.shape[0]
    columns_let = num_gagong_bf.shape[1]
    Yij_shell_let = Yij_shell.copy()

    Q_np = Q_let.copy()
    
    # Conversion of (1,0) binary data into (1,-1) binary data (NaN is transformed to zero)
    # Refinement for avoiding 'divided by zero' error
    num_gagonged_bf = np.where(num_gagong_bf == 0.01, -0.99, num_gagong_bf)
    num_gag_pd = pd.DataFrame(num_gagonged_bf)
    num_gag_fna = num_gag_pd.fillna(0)
    num_gagonged_np = num_gag_fna.to_numpy()
    
    p_bfsum = Shell_gagong(num_gagonged_np, Q_np)       # the numberator before sum of the formula above

#---------------------------------------------------simple sum up ------------- Normalization down ---------------------------------------
    
    # generation of denominator of the formula above
    
    denomin = []
    for i in range(rows):
        bf_Qsam = Yij_shell[i] * Q_let
        af_Qsam = bf_Qsam.sum()
        denomin.append(af_Qsam)

    P2_carrier = p_bfsum.copy()          # 3-Rank Tensor
    
    # Ingredient of pseudo-probability
    for i in range(rows_let):
        if denomin[i] == 0:
            P2_carrier[i] = 0 * P2_carrier[i] # Get rid of the information of examinees who solved only one item.
        else:
            P2_carrier[i] = P2_carrier[i] / denomin[i]
    
    return P2_carrier
        

In [ ]:
# Generation of pseudo-probability is accomplished in the end.
# Gagong_data is of pandas and Q_let is of numpy.
def answer_covari_afsum(gagong_data, Q_let):
    
    # collection of the whole ingredient
    p_bfsum = answer_covari_bfsum(gagong_data, Q_let)
    gagong_np = gagong_data.to_numpy()
    rows_let = gagong_np.shape[0]
    columns_let = gagong_np.shape[1]
    
    covari_ini = p_bfsum.sum(axis=2)
    covari_mid = covari_ini.sum(axis=1)
    covari_carry = np.reshape(covari_mid, (rows_let, 1))  # keep the vertical shape

    # pseudo-probability of range between 0 and 1

    mid_result = (49/98.01) * (covari_carry) + 0.5  # refinement avoding 'divided by zero' error
    
    # refinement avoding 'divided by zero' error
    scarub = np.where(mid_result > 0.99, 0.99, mid_result)
    scourge = np.where(scarub < 0.01, 0.01, scarub)
    P2_result = scourge
        
    return P2_result       # pseudo-probability of numpy form

In [ ]:
# The function to calculate the derivative of KLD by Q
# Gagong_data is of pandas the others are of numpy.
def Q_deriv(alp1, alp2, d_let, tht1, tht2, Q_let, gagong_data):

    num_gagong_bf = gagong_data.to_numpy()
    rows_let = num_gagong_bf.shape[0]
    columns_let = num_gagong_bf.shape[1]
    Yij_shell_let = Yij_shell.copy()

    Q_np = Q_let.copy()             
    Q_nuul = Q_halves.copy()       # The initialized Q matrix of Universality
    
    # Conversion of (1,0) binary data into (1,-1) binary data (NaN is transformed to zero)
    # Refinement for avoiding 'divided by zero' error
    num_gagonged_bf = np.where(num_gagong_bf == 0.01, -0.99, num_gagong_bf)
    num_gag_pd = pd.DataFrame(num_gagonged_bf)
    num_gag_fna = num_gag_pd.fillna(0)
    num_gagonged_np = num_gag_fna.to_numpy()
    
    p_bfsum_nossi = Shell_gagong(num_gagonged_np, Q_nuul)
    p_bfsum = Shell_gagong(num_gagonged_np, Q_np)         # Before calculation

#---------------------------------------------------division line-------------------------------#
    
    # generation of denominator of the formula above
    denomin = []
    for i in range(rows):
        bf_Qsam = Yij_shell[i] * Q_let
        af_Qsam = bf_Qsam.sum()
        denomin.append(af_Qsam)

    P2_carrier1 = p_bfsum_nossi.copy()
    P2_carrier20 = p_bfsum.copy()          # 3-Rank Tensor

    # The 1st term of the numerator
    for i in range(rows_let):
        if denomin[i] == 0:
            P2_carrier1[i] = 0 * P2_carrier1[i]
        else:
            P2_carrier1[i] = P2_carrier1[i] / denomin[i]

    # The 2nd term of the numerator
    for i in range(rows_let):
        if denomin[i] == 0:
            P2_carrier20[i] = 0 * P2_carrier20[i]
        else:
            P2_carrier20[i] = P2_carrier20[i] / (denomin[i] * denomin[i])
            
    covari2_ini = P2_carrier20.sum(axis=2)
    covari2_mid = covari2_ini.sum(axis=1)
    P22_part = np.reshape(covari2_mid, (rows_let, 1))      # keep the vertical shape
    
    P2_list = []
    for i in range(rows_let):
        carrier = Yij_shell_let[i] * P22_part[i]
        P2_list.append(carrier)
    
    P2_carrier2 = np.array(P2_list)
    
    return P2_carrier1, P2_carrier2 # former: the 1st term, latter: the 2nd term of the numerator
        

In [ ]:
# The function to sum all the ingredient of the formula above in the end
# Gagong_data is of pandas the others are of numpy.
def Q_learn(alp1, alp2, d_let, tht1, tht2, Q_let, gagong_data):
    
    Q_np_test = Q_let.copy()               # Matrix to be learned
    gagonged_data = gagong_data.to_numpy()
    rows_let = gagonged_data.shape[0]
    columns_let = gagonged_data.shape[1]

    # the chain of the derivative: 3-Rank Tensor form
    P2_mu = answer_covari_afsum(gagong_data, Q_np_test)
    Normed_Y = (49/98.01) * (Q_deriv(alp1, alp2, d_let, tht1, tht2, Q_let, gagong_data)[0] - Q_deriv(alp1, alp2, d_let, tht1, tht2, Q_let, gagong_data)[1])

#---------------------------------------------------division line----------------------------------------------------------------------------------#
    
    # common part
    com_pt = preprocess_diff(alp1, alp2, d_let, tht1, tht2, gagong_data)

    # calculation start
    common_unit_np = com_pt * alp1                               # 2-dimensional Matrix
        
    common_unit_T = np.transpose(common_unit_np)                 # mu for axis=1; in order to link mu with 3-Rank Tensor
    decoy_1st = pd.DataFrame(common_unit_T)
    decoy_2nd = decoy_1st.fillna(0)
    common_unit = decoy_2nd.to_numpy()                           

#-----------------------------------Now, it's time to build a 4-Rank tensor ------------------------------------------------------------------------#
    
    P_hat_list = []                              # Initialize the list to store a 4-Rank Tensor
    P_hat_3D = []                                # Initialize the list to store a 3-Rank Tensor
    carrier_2D = []

    for i in range(columns_let):
        for j in range(columns_let):
            for mu in range(rows_let):
                carrier = common_unit[:, mu] * Normed_Y[mu, i, j] / (P2_mu[mu, 0] * (1 - P2_mu[mu, 0]))
                carrier_2D.append(carrier)
            P_hat_3D.append(carrier_2D)          # combination of mu and k components is added.
            carrier_2D = []                      # Reset the 2D matrix
        P_hat_list.append(P_hat_3D)              # complete the ith component
        P_hat_3D = []                            # Reset the 3-Rank Tensor
        
    P_hat_np = np.array(P_hat_list)              # complete the 4-Rank Tensor
    
    #Then, sum it up in terms of k and mu axes.
    # KLD Gradient Discent
    Q_pre = P_hat_np.sum(axis=3)                 # sum it up in mu axis
    Q_presum = Q_pre.sum(axis=2)                 # sum it up in k axis
    
    # Final Gradient Descendent: update
    Q_med = Q_np_test - A * Q_presum
    np.fill_diagonal(Q_med, 0)
    Q_result = Q_med/(2 * Q_med.mean())          # Normalization: the average of all the component should be 0.5.

    return Q_result                             # The result is yielded in 2D matrix of numpy form.


In [ ]:
# the function to update theta_2
# Only the gagong_data is given in pandas.
# Theta_2 is updated via the imaged process above.
def set_theta_Q(gagong_data, Q_let):
    
    rate_result = answer_covari_afsum(gagong_data, Q_let)
    
    theta_result = np.log((rate_result)/(1 - rate_result))
    
    return theta_result       # The result is yielded in numpy form.

In [ ]:
# the function to calculate KLD
# Train_gagong_let is of pandas the others are of numpy.
def set_D_KL(alpha_let, alpha2le, d_let, theta_let, theta2le, train_gagong_let):
    
    num_gagong = train_gagong_let.to_numpy()
    P_imu = expect_model(alpha_let, alpha2le, d_let, theta_let, theta2le, train_gagong_let)
    Q_imu = num_gagong.copy()
    
    KLD_imu_np = Q_imu * np.log((Q_imu) / (P_imu)) + (1 - Q_imu) * np.log((1 - Q_imu)/(1 - P_imu))
    
    # Get rid of Missing Data
    KLD_imu_df = pd.DataFrame(KLD_imu_np)
    KLD_shuttle = KLD_imu_df.fillna(0)
    KLD_imu = KLD_shuttle.to_numpy()
    
    D_KL_mu = KLD_imu.sum(axis=1)
    D_KL = D_KL_mu.sum(axis=0)
    
    return D_KL

In [ ]:
# Iteration function for model optimization

def opt_model(alpha_let, d_let, theta_let, theta2le, Q_let, train_gagong_let, test_gagong_let, num_iter):

    # Initialization of parameters and variables
    alpha1_test = alpha_let.copy()
    alpha2_test = alpha_let.copy()
    d_test = d_let.copy()
    Q_test = Q_let.copy()
    theta1_test = theta_let.copy()
    theta2_test = theta2le.copy()

    KLD_train = set_D_KL(alpha1_test, alpha2_test, d_test, theta1_test, theta2_test, train_gagong_let)
    KLD_Trains = []
    KLD_Trains.append(KLD_train)
    
    KLD_testset = set_D_KL(alpha1_test, alpha2_test, d_test, theta1_test, theta2_test, test_gagong_let)
    KLD_Tests = []
    KLD_Tests.append(KLD_testset)
    
    for k in tqdm(range(num_iter)):

        # alpha1 update
        alpha1_carrier = set_alpha(alpha1_test, alpha2_test, d_test, theta1_test, theta2_test, train_gagong_let)
        alpha1_test = alpha1_carrier

        # alpha2 update
        alpha2_carrier = set_alpha(alpha2_test, alpha1_test, d_test, theta2_test, theta1_test, train_gagong_let)
        alpha2_test = alpha2_carrier

        # d update
        delta_carrier = set_delta(alpha1_test, alpha2_test, d_test, theta1_test, theta2_test, train_gagong_let)
        d_test = delta_carrier

        # Q update
        Q_carrier = Q_learn(alpha2_test, alpha1_test, d_test, theta2_test, theta1_test, Q_test, train_gagong_let)
        Q_test = Q_carrier

        # theta1 update
        theta1_carrier = update_theta(alpha1_test, alpha2_test, d_test, theta1_test, theta2_test, train_gagong_let)
        theta1_test = theta1_carrier

        # theta2 update
        theta2_carrier = set_theta_Q(train_gagong_let, Q_test)
        theta2_test = theta2_carrier

        # calculation of Kullback-Leibler Divergence
        KLD_carrier = set_D_KL(alpha1_test, alpha2_test, d_test, theta1_test, theta2_test, train_gagong_let)
        KLD_testset = set_D_KL(alpha1_test, alpha2_test, d_test, theta1_test, theta2_test, test_gagong_let)
        #print("Kullback-Leibler Divergence of the Iteration %d: " % (k+1), KLD_carrier)

        # Determination whether the iteration keeps or not
        if (k < num_iter - 1) and (KLD_carrier < KLD_train):
            KLD_train = KLD_carrier
            KLD_Trains.append(KLD_train)        # store KLD of trian set
            KLD_Tests.append(KLD_testset)       # store KLD of test set
        elif k == num_iter - 1:                # the final iteration
            KLD_train = KLD_carrier
            KLD_Trains.append(KLD_train)        # store of KLD of train set
            KLD_Tests.append(KLD_testset)       # store of KLD of test set
            print("Final Kullback-Leibler Divergence: ", KLD_train)

        else:
            print("Final Kullback-Leibler Divergence: ", KLD_train)

            break
            
    return alpha1_test, alpha2_test, d_test, theta1_test, theta2_test, Q_test, KLD_Trains, KLD_Tests

## Now, it is very time to play the real game!

In [ ]:
from tqdm import tqdm

### The Body of the Model Optimization

In [ ]:
albetheQKLD = []
num_iter = 0
#train_trial = []
#train_trial.append(train_gagongs[0])

#for gagong_carrier in train_trial:
for gagong_carrier in train_gagongs:
    carrier_shell = []

    num_dfdf = gagong_carrier.copy()
    p_df = num_dfdf.copy()
    num_np = num_dfdf.to_numpy()

    # theta_1 initialization
    row_pre = p_df.mean(axis=1)
    row_prob_1 = row_pre.to_numpy()
    row_prob = np.reshape(row_prob_1, (rows,1))

    theta_1 = np.log(row_prob/(1-row_prob))

    # d initialization
    col_pre = p_df.mean(axis=0)
    col_prob_1 = col_pre.to_numpy()
    col_prob = np.array([col_prob_1])
    d0 = np.log(col_prob/(1-col_prob))
    d = np.mean(d0) - d0

    # alpha_1 and alpha_2 initialization
    alpha = np.ones((1,columns))
    
    A = 0.005            # learning rate

    # transformation of (1,0) binary responses into (1,-1) binary responses
    num_exp1 = num_np.copy()
    num_exp2 = np.where(num_exp1 == 0.01, -0.99, num_exp1) # transformation

    num_exp_df = pd.DataFrame(num_exp2)
    num_exp_af = num_exp_df.fillna(0)                 # get rid of NaN
    num_exp_np = num_exp_af.to_numpy()

    # Q initialization
    Q_np_ini = np.ones((columns, columns))
    np.fill_diagonal(Q_np_ini, 0)
    Q_halves = Q_np_ini / 2

    # theta_2 initialization
    shell_list = []

    for i in range(rows):
        garo_pre = num_exp_np[i, :]
        garo_T = np.reshape(garo_pre, (columns, 1))   # vertial vector form
        sero = garo_T.copy()
        garo = np.transpose(garo_T)
        carrier = sero * garo                   
        np.fill_diagonal(carrier, 0)                 # off-diagonal

        shell_list.append(carrier)

    shell_ini = np.array(shell_list)          # initial combination of Y_iY_j

    # the reference to indicate the location of solved items
    Y_solved0 = num_np.copy()
    Y_solved1 = np.where(Y_solved0 == 0.01, 1, Y_solved0)
    Y_solved2 = np.where(Y_solved1 == 0.99, 1, Y_solved1)
    Y_pd = pd.DataFrame(Y_solved2)
    Y_fna = Y_pd.fillna(0)         # set NaN as zero
    Y_solved = Y_fna.to_numpy()

    Yij_solved = []

    for i in range(rows):
        garo_pre = Y_solved[i, :]
        garo_T = np.reshape(garo_pre, (columns, 1))
        sero = garo_T.copy()
        garo = np.transpose(garo_T)
        carrier = sero * garo                   
        np.fill_diagonal(carrier, 0)            

        Yij_solved.append(carrier)

    Yij_shell = np.array(Yij_solved)
    
    denominator = []
    for i in range(rows):
        bf_Qsum = Yij_shell[i] * Q_halves
        af_Qsum = bf_Qsum.sum()
        denominator.append(af_Qsum)        # generation of the denominator

    P_carrier = []     # basket for initial pseudo-probability
    for i in range(rows):
        garo_pre = num_exp_np[i, :]
        garo = np.reshape(garo_pre, (1, columns))
        sero_T = np.copy(garo)
        sero = np.transpose(sero_T)

        vectorman1 = sero * garo
        vectorman11 = Q_halves * vectorman1
        vectorman111 = vectorman11.sum(axis=1)
        vectorman2 = vectorman111.sum(axis=0)

        if denominator[i] == 0:
            P_mu = 0
        else:
            P_mu = vectorman2 / denominator[i]

        P_carrier.append(P_mu)

    P_norm = np.array(P_carrier)

    theta_pre = (49/98.01) * (P_norm) + 0.5    # final form of pseudo-probability initialization

    # final initialization of theta_2
    theta1_bfT = np.log(theta_pre / (1 - theta_pre))
    theta_2 = np.reshape(theta1_bfT, (rows,1))

    # initialization of the probability distribution of the model
    exp1 = alpha * theta_1
    exp2 = alpha * theta_2
    ex_prob = np.exp(exp1 + exp2 - d)/(1+np.exp(exp1 + exp2 - d))

    ex_prob_real = ex_prob.copy()

    for n in range(ex_prob.shape[0]):        # reflect the distribution of NaN
        for m in range(ex_prob.shape[1]):
            if np.isnan(num_np[n][m]):
                ex_prob_real[n][m] = np.nan

    # KLD of each response
    KLD_indiv = num_np * np.log(num_np / ex_prob_real) + (1 - num_np) * np.log((1 - num_np) / (1 - ex_prob_real))

    # get rid of missing values
    KLD_indiv_df = pd.DataFrame(KLD_indiv)
    KLD_NaNga_df = KLD_indiv_df.fillna(0)
    KLD_NaNga_np = KLD_NaNga_df.to_numpy()

    # KLD initialization
    KLD_RowSum = np.sum(KLD_NaNga_np, axis=1)
    KLD_TotalSum_np = np.sum(KLD_RowSum, axis=0)

    # Model Optimization Start
    alpha1_mod, alpha2_mod, d_mod, theta1_mod, theta2_mod, Q_mod, KLDs_mod, KLDs_test_mod = opt_model(alpha, d, theta_1, theta_2, Q_halves, p_df, test_gagongs[num_iter], 20)

    # save for further analysis
    carrier_shell.append(alpha1_mod)       # 0
    carrier_shell.append(alpha2_mod)       # 1
    carrier_shell.append(d_mod)            # 2
    carrier_shell.append(theta1_mod)       # 3
    carrier_shell.append(theta2_mod)       # 4
    carrier_shell.append(Q_mod)            # 5
    carrier_shell.append(KLDs_mod)         # 6
    carrier_shell.append(KLDs_test_mod)    # 7

    albetheQKLD.append(carrier_shell)
    num_iter += 1


In [ ]:
print(albetheQKLD)
print(albetheQKLD[0][7])

In [ ]:
num_save = 0
for carrying in albetheQKLD:
    num_save+=1
    
    KLD_trained = carrying[6]
    KLD_tested = carrying[7]
    
    KLDs_train_pd = pd.DataFrame(KLD_trained)
    KLDs_test_pd = pd.DataFrame(KLD_tested)

    KLDs_train_pd.to_csv("MIRT{0}_KLDs_50_0.1train_0705.csv".format(num_save))
    KLDs_test_pd.to_csv("MIRT{0}_KLDs_50_0.1test_0705.csv".format(num_save))

## Test set vs Train set and Save the Final Data

In [ ]:
def expect_simple_cal(alpha1_let, alpha2_let, d_let, theta1_let, theta2_let):
    exp1 = alpha1_let * theta1_let
    exp2 = alpha2_let * theta2_let

    cal2 = np.exp(exp1 + exp2 - d_let)/(1+np.exp(exp1 + exp2 - d_let))
    
    if cal2 >= 0.99:
        cal2 = 0.99
    elif cal2 <= 0.01:
        cal2 = 0.01

    cal_result = cal2 

    return cal_result   # return in number

In [ ]:
basket_final = []
num_unit = 0

for gagong_unit in test_gagongs:
    # pick up a basket of parameters
    basket_picks = albetheQKLD.copy()[num_unit]

    # update the index
    num_unit += 1

    # distribution of numbers for thetas
    theta1_fin_df = pd.DataFrame(basket_picks[3])
    theta2_fin_df = pd.DataFrame(basket_picks[4])

    # namtaging of alphas and d
    alpha1_fin_df = pd.DataFrame(basket_picks[0])
    alpha2_fin_df = pd.DataFrame(basket_picks[1])
    d_fin_df = pd.DataFrame(basket_picks[2])

    alpha1_fin_df.columns = fil4.columns.to_list()
    alpha2_fin_df.columns = fil4.columns.to_list()
    d_fin_df.columns = fil4.columns.to_list()

    # store the all the information about test set
    threshed_theta = []

    # Set the coordinate
    coord_pd = pd.read_csv("UIRT_0.1testset_{0}_0701.csv".format(num_unit))
    coord_pd.drop(['Unnamed: 0'], axis=1, inplace=True)
    sampl_len = coord_pd.shape[1]
    
    for th in range(sampl_len):
        piece_th = []
        
        coord_x = int(coord_pd.loc[0][th])
        coord_col = coord_pd.loc[1][th]
        correctness = float(coord_pd.loc[2][th])

        theta1_piece = theta1_fin_df[0][coord_x]
        alpha1_piece = alpha1_fin_df.loc[0][coord_col]
        theta2_piece = theta2_fin_df[0][coord_x]
        alpha2_piece = alpha2_fin_df.loc[0][coord_col]
        d_piece = d_fin_df.loc[0][coord_col]
        expect_cal = expect_simple_cal(alpha1_piece, alpha2_piece, 
                                       d_piece, theta1_piece, theta2_piece)

        piece_th.append(coord_x)         # 0
        piece_th.append(coord_col)       # 1
        piece_th.append(correctness)     # 2
        piece_th.append(theta1_piece)    # 3
        piece_th.append(alpha1_piece)    # 4
        piece_th.append(theta2_piece)    # 5
        piece_th.append(alpha2_piece)    # 6
        piece_th.append(d_piece)         # 7
        piece_th.append(expect_cal)      # 8
        threshed_theta.append(piece_th)  # th

    #print(threshed_theta)       # 순서: 학생 index, 문제 index, 문제 정오, 
                                        # theta, alpha, beta, 모델계산값
    
    threshed_pick = threshed_theta.copy()

    # Judgment whether the imputations correct or not
    stud_info_simple = []
    num_R = 0            # number of right imputation
    num_W = 0            # number of wrong imputation
    num_tot = 0          # number of total items in the test set

    for student in threshed_pick:
        carrier_bot = []
        data_real = student[2]
        data_cal = student[8]
        data_jud = 0
        data_RW = ''                       # Initialization

#        if data_cal >=0.7:                 # rounding off to the nearest integer
#            data_jud = 0.99
        if data_cal >=0.5:                 # rounding off to the nearest integer
            data_jud = 0.99
#        elif data_cal <= 0.3:
#            data_jud = 0.01
#        else:
#            data_jud = 0.5
        else:
            data_jud = 0.01

        if data_real == data_jud:
            data_RW = 'O'                   # 'O' represents the model imputed correctly.
            num_R += 1
            num_tot += 1
        else:
            data_RW = 'X'                   # 'X' represents the model imputed incorrectly.
            num_W += 1
            num_tot +=1

        carrier_bot.append(student[0])      # examinee number
        carrier_bot.append(student[1])      # item number
        carrier_bot.append(data_RW)         # judgment
        stud_info_simple.append(carrier_bot)

    print("Accordance Ratio of Ising MIRT: {0}".format(num_R/num_tot * 100))